In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Lasso, Ridge, LassoCV,LinearRegression

import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 30)

#sns.set_style("whitegrid")
#plt.style.use('bmh')
plt.style.use('seaborn-whitegrid')

# this allows plots to appear directly in the notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
def get_rmsle(y_pred, y_actual):
    diff = np.log(y_pred + 1) - np.log(y_actual + 1)
    mean_error = np.square(diff).mean()
    return np.sqrt(mean_error)

rf_pred = pd.read_csv('output2/submit_rf_val_20211025_3.csv', index_col=0)
gbm_pred = pd.read_csv('output2/submit_gbm_val_20211025_3.csv', index_col=0)
ridge_pred = pd.read_csv('output2/submit_ridge_val_20211025_3.csv', index_col=0)
#lasso_pred = pd.read_csv('output2/submit_lasso_val_20211022_1.csv', index_col=0)
ridge_actual = pd.read_csv('output2/ridge_actual.csv', index_col=0)

# the blend gives a better score on the leaderboard, even though it does not on the validation set
best_score = 1
for i in np.arange(0, 1, 0.01):
    for j in np.arange(0, 1, 0.01):
        #for k in np.arange(0, 1, 0.01):
        y_pred = np.round(i*np.array(rf_pred) + j*np.array(ridge_pred) + (1-i-j)*np.array(gbm_pred))#k*np.array(lasso_pred) + (1-i-j-k)*np.array(gbm_pred))
        score = get_rmsle(y_pred,np.array(ridge_actual))
        if score < best_score:
            best_score = score
            best_rate = {'i':i,'j':j}#'k':k}

print(f'best_score: {best_score}, best_rate: {best_rate}')


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


best_score: 0.3411892036783972, best_rate: {'i': 0.27, 'j': 0.12}


In [3]:
rf_pred = pd.read_csv('output2/submit_rf_20211025_3.csv', index_col=0)
gbm_pred = pd.read_csv('output2/submit_gbm_20211025_3.csv', index_col=0)
ridge_pred = pd.read_csv('output2/submit_ridge_20211025_3.csv', index_col=0)
#lasso_pred = pd.read_csv('output2/submit_lasso_20211022_1.csv', index_col=0)

y_pred = np.round(best_rate['i']*np.array(rf_pred) + best_rate['j']*np.array(ridge_pred) + (1-best_rate['i']-best_rate['j'])*np.array(gbm_pred))#best_rate['k']*np.array(lasso_pred) + (1-best_rate['i']-best_rate['j']-best_rate['k'])*np.array(gbm_pred))

temp_pred = pd.read_csv('output2/submit_rf_20211015_1.csv')
# output predictions for submission
submit_manual_blend_df = temp_pred.copy()#[['datetime', 'count']].copy()
submit_manual_blend_df['count'] = y_pred
submit_manual_blend_df.to_csv('output2/submit_manual_blend_20211025_3.csv', index=False)
